In [28]:
from dotenv import dotenv_values
import os

env_vars = dotenv_values('../.env')
os.environ['OPENAI_API_KEY'] = env_vars.get('OPENAI_API_KEY')

In [39]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

In [44]:
loader = DirectoryLoader("./data/TS_1.판결문/1.Training/원천데이터/TS_1.판결문/02.형사/2021", glob="**/*.xml")
docs = loader.load()

In [56]:
docs_cnt = []
for item in range(len(docs)):
    num = len(docs[item].page_content)
    docs_cnt.append(num)

In [58]:
docs_cnt

[1097,
 3962,
 3263,
 4054,
 1344,
 1107,
 1880,
 1401,
 2412,
 4137,
 1917,
 15201,
 4916,
 1983,
 2606,
 7354,
 1736,
 2326,
 2426,
 1603,
 2415,
 2452,
 16618,
 7937,
 3006,
 2469,
 2713,
 2287,
 1308,
 3606,
 2722,
 3106,
 2711,
 4698,
 5051,
 5506,
 6218,
 2002,
 2596,
 1375,
 4008,
 3449,
 3145,
 5436,
 9160,
 2638,
 1810,
 2902,
 5151,
 6965,
 1991,
 2043,
 2759,
 2586,
 3154,
 860,
 1325,
 3173,
 1320,
 1459]

In [11]:

model_name = "jhgan/ko-sroberta-multitask" # (KorNLU 데이터셋에 학습시킨 한국어 임베딩 모델)
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/home/crysis/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
vectorstores = Chroma.from_documents(docs, embedding_model)
retriever = vectorstores.as_retriever()

In [19]:
retrieved_docs = retriever.invoke(
    "절도 사건에 관한 판례를 알려줘"
)

In [20]:
len(retrieved_docs)

4

In [27]:
print(retrieved_docs[0].page_content)


		
판결문_형사 PDF		
		


		
2021고단2704.pdf		
		


		

, 1 ,

서 울 남 부 지 방 법 원

판          결

사       건 2021고단2704  절도

피  고  인 A

검       사 00(기소), 00(공판)

변  호  인 변호사 00(국선)

판 결 선 고 2021. 8. 26.

  

주       문

피고인을 판시 제1 죄에 대하여 벌금 20만 원에, 판시 제2 죄에 대하여 벌금 30만 원

에 각 처한다.

피고인이 위 벌금을 납입하지 아니하는 경우 10만 원을 1일로 환산한 기간 피고인을 

노역장에 유치한다.

벌금에 상당한 금액의 가납을 명한다. 

  

이       유

범 죄 사 실

  피고인은 2019. 12. 19. 인천지방법원 부천지원에서 절도죄로 징역 6월에 집행유예 

2년을 선고받고 2019. 12. 27. 그 판결이 확정되었다. 

1. 2019. 10. 19.자 절도



, 2 ,

  피고인은 2019. 10. 19. 11:10경 0000에 있는 피해자 C이 운영하는 D 0000

점에서, 주위의 감시가 소홀한 틈을 타 그곳 판매대에 있던 시가 합계 18,630원 상

당의 식료품을 소지하고 있던 가방에 넣어 가지고 갔다.

  이로써 피고인은 피해자의 재물을 절취하였다.

2. 2021. 1. 1.자 절도

  피고인은 2021. 1. 1. 11:00경 0000 분리수거장에서, 주위의 감시가 

소홀한 틈을 타 피해자 E 아파트 소유인 시가 불상의 소주병 2개를 가지고 갔다.

  이로써 피고인은 피해자의 재물을 절취하였다.

증거의 요지

1. 피고인의 법정진술

1. F에 대한 경찰 진술조서

1. G의 진술서

1. 영수증, 재활용품 매각 계약서

1. 각 피해품 및 현장 CCTV 사진

1. 판시 전과 : 범죄경력등조회회보서, 수사보고(관련전과 확인), 판결문 및 약식명령문 

법령의 적용

1. 범죄사실에 대한 해당법조 및 형의 선택

   각 

In [63]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

template = """다음과 같은 맥락을 사용하여 마지막 질문에 대답하십시오.
만약 답을 모르면 모른다고만 말하고 답을 지어내려고 하지 마십시오.
맥락: {context}
질문: {question}
도움이 되는 답변:"""
rag_prompt_custom = PromptTemplate.from_template(template)

In [64]:
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm

In [66]:
rag_chain.invoke("마약 사건에 관한 판례 번호와 해당 판례의 판결선고일을 알려줘")

AIMessage(content='1. 판례 번호: 2021고합31\n   판결선고일: 2021. 5. 7.\n\n2. 판례 번호: 2021고합25\n   판결선고일: 2021. 5. 7.\n\n3. 판례 번호: 2021고합174\n   판결선고일: 2021. 8. 20.\n\n4. 판례 번호: 2021고단1951\n   판결선고일: 2021. 8. 26.')

In [67]:
rag_chain.invoke("절도 사건에 관한 판례 번호와 해당 판례의 판결선고일을 알려줘")

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 23858 tokens. Please reduce the length of the messages.